## Reactive Functions & Expressions

In the [Dependencies and Watchers](Dependencies_and_Watchers.ipynb) guide we discovered how to express dependencies and write callbacks that are invoked when parameter values change. This imperative style of expressing dynamic behavior is powerful, but can create in complex code that is hard to read and maintain. Param 2.0 introduces a new set of primitives that make it possible to express complex dependencies in a declarative form, resulting in reactive functions and expressions that are re-evaluated automatically when required.

The reactive programming model is one you might be familiar with from spreadsheets like Excel, where formulas can reference cells or ranges and dynamically (or more precisely, _reactively_) recompute when the inputs to a formula changes. In Param, Parameter objects correspond to a spreadsheet cell formula's inputs or references, and reactive expressions correspond to the formula itself. `param.bind` also allows the creation of a reactive function with arbitrary inputs.

This user guide is structured as two main sections:

- [Reactive Functions](#Reactive-Functions): Using `param.bind` to declare functions that react when their inputs change.
- [Reactive Expressions](#Reactive-Expressions): Using `param.reactive` (or `.reactive()` on Parameter objects) to wrap ordinary objects and/or parameters in a proxy that acts like the underlying object but reacts when an input changes.

## Reactive Functions

To get started, let's first write a simple, non-reactive function to add two arguments:

In [ ]:
def add(a, b):
    return a + b

add(3,7)

Now, let's make a `Parameterized` class with some `Parameters`, here named `a` and `b`, that we'll want to add together reactively. We will also import `param.ipython` to install support for displaying "live" reactive components in an IPython or Jupyter notebook, re-rendering themselves when the inputs change.

In [ ]:
import param
import param.ipython

class Parameters(param.Parameterized):
    
    a = param.Number(1)

    b = param.Number(0)

    run = param.Event()
    
p = Parameters()

Ok, now we can use `param.bind` to "bind" parameters `a` and `b` to create a reactive function:

In [ ]:
reactive_add = param.bind(add, p.param.a, p.param.b)

reactive_add

As you can see, `reactive_add` works just like `add`, in that it adds two arguments, but in this case it's taking the value of the `a` and `b` Parameters of `p`. Parameter `a` has been "bound" to the first argument and `b` to the second, and if either of them changes, the result changes. So if we change `p.a` to 5, the output above changes immediately (it "reacts").

In [ ]:
p.a += 4

We can also call the reactive function explicitly to return the current result as a concrete, no longer reactive value:

In [ ]:
reactive_add()

The difference between `reactive_add` and `reactive_add()` is that the first one is a function, whose display will automatically update in IPython/Jupyter thanks to the extension loaded above, while the second is a specific number (the result of calling that function a single time, never to be updated further):

In [ ]:
print(type(reactive_add), type(reactive_add()))

`param.bind` follows the semantics of Python's `functools.partial`, and so if you only partially bind the required arguments, you'll get a function of the remaining arguments:

In [ ]:
add_b = param.bind(add, p.param.a)
add_b

In [ ]:
add_b(5)

Note that you can bind any accepted type to make a reactive function, not just Parameters, but static values won't trigger reactive updates (here 38 will always be the same value, while the result will depend on the current value of `p.param.a`).

In [ ]:
param.bind(add, p.param.a, 38)

## Reactive Expressions

While reactive functions are very useful and allow writing arbitrarily complex logic, they still require writing a Python function definition, which can be verbose to write and difficult to read. With a `reactive` expression instead of an explicitly defined function, you can wrap any object or parameter value and apply operations on it, just as if you are working with the actual object, but now with reactive outputs. In other words, the reactive expression acts as a proxy for the underlying value, while supporting (almost) all operations that can be performed with the original object.

### Using Parameters

As an example, let's create reactive proxies for the `a` and `b` parameters and add them together:

In [ ]:
q = Parameters()

expr = q.param.a.reactive() + q.param.b.reactive() + 3

expr

The resulting reactive expression now reflects the result of this operation and will update automatically when one of the inputs to the operation changes, e.g. if we update parameter `a`:

In [ ]:
q.a += 2

### Resolving the expression

Reactive objects generally just provide whatever API the underlying object has, but there are a few extra reactive-specific methods also provided. In order to avoid any clashes between the namespace of the reactive expression and the object it is wrapping, the extra methods are in a special namespace called `.rx`.

For instance, to resolve the current value of the expression into the current value as a static (non-reactive) object, we can call `.rx.resolve()`:

In [ ]:
expr.rx.resolve()

The value here is no longer a reactive expression, and so it will not update when `a` or `b` changes:

In [ ]:
q.b += 2

### Using literal objects as inputs

The convenient `param.reactive` function lets you make just about _anything_ reactive, without having to first define a new Parameterized object with explicit Parameters. E.g. we can create a `reactive` object from a static, literal value, such as a string:

In [ ]:
string_template = param.reactive('Hello {name}!')

string_template

The `reactive` object now acts like a string so we can use the `.format` method to fill in the `string_template` with another reactive expression:

In [ ]:
name = param.reactive('world')

str_expr = string_template.format(name=name)

str_expr

### Setting the input value

To update the input to a `reactive` object we can use the `.rx.set()` method:

In [ ]:
name.rx.set('there')

str_expr.rx.resolve()

### Special Methods & Limitations

A `reactive` proxy behaves like the underlying object it is wrapping, but only to the extent that Python allows. Certain operations cannot be implemented in this way, e.g. Python will not allow the `len` operation to return anything but a integer and the `is` statement always checks the immediate identity of its two operands. Therefore `reactive` implements certain operations as special methods that exist on the `.rx` namespace alongside `rx.set` and `rx.resolve`:

- `.rx.bool()`: Reactive version of `bool()`, casting the output value to a Boolean.
- `.rx.is_()`: Reactive version of `is`, testings the object identity against another object.
- `.rx.is_not()`: Reactive version of `is not`, testing the absence of object identity with another object.
- `.rx.len()`: Reactive version of `len()`, returning the length of the expression
- `.rx.pipe()`: Applies the given function (with static or reactive arguments) to this object.
- `.rx.when()`: Generates a new expression that only updates when the provided dependency updates.
- `.rx.where()`: Returns either the first or the second argument, depending on the current value of the expression.

#### `.rx.bool()`

Casts the current value to a Boolean True or False value:

In [ ]:
param.reactive(1).rx.bool()

#### `.rx.is_(arg)`

Checks the identity of the current value is the same as the argument to `.is_`

In [ ]:
param.reactive(None).rx.is_(None)

#### `.rx.is_not(arg)`

Checks the identity of the current value is not the same as the argument to `.is_not`

In [ ]:
param.reactive(None).rx.is_not(None)

#### `.rx.len()`

Returns the length of the object

In [ ]:
param.reactive([1, 2, 3]).rx.len()

#### `.rx.pipe(func, *args, **kwargs)`

Pipes the current value into a function as the first argument, passing in additional positional and keyword arguments if provided:

In [ ]:
param.reactive(1).rx.pipe(add, 2)

In [ ]:
param.reactive(8).rx.pipe(str)

#### `.rx.when(*conditions)`

Useful when creating UIs to declare that the expression should only update when some other parameter changes, e.g. when a user clicks a button or triggers an expensive operation through some other mechanism.

For instance, let's say we have some expensive function (here simulated using `time.sleep`). First we bind parameters `a` and `b` to this function and create a reactive expression from this function.

In [ ]:
import time

def expensive_function(a, b):
    time.sleep(2)
    return a * b

p = Parameters()

expensive_expr = param.bind(expensive_function, p.param.a, p.param.b).reactive()

In [ ]:
e = param.bind(expensive_function, p.param.a, p.param.b)

In [ ]:
e

In [ ]:
p.param.update(a=7, b=5)

The problem we face is that if we use this `expensive_expr` whenever `a` **or** `b` are changed, then the expensive computation gets triggered *twice* if we want to change both `a` _and_ `b`. We could use `p.param.update()` to change them in bulk in this particular case, but since that's not always sufficient, here we'll gate the computation behind a third variable, e.g. the `run` `Event` parameter on the `Parameters` class.

In [ ]:
gated_expr = expensive_expr.rx.when(p.param.run)

gated_expr

We can now safely change variables `a` and `b` separately without triggering the computation:

In [ ]:
p.a = 2
p.b = 4

gated_expr.rx.resolve()

But when we trigger the `run` parameter the expression will re-compute:

In [ ]:
p.param.trigger('run')

gated_expr.rx.resolve()

#### `.rx.where(x, y)`

Where allows writing ternary conditions using the reactive paradigm. Ordinarily you can write something like this:

```python
value_a if some_condition else value_b
``` 

to return `value_a` or `value_b` depending on some condition. However, Python does not allow overriding `if`, so `if`  statements are not re-evaluated when the condition changes, and instead we have to rewrite this case using `where`.

First we will declare a condition, which here is simply a `reactive` expression wrapping a Boolean value:

In [ ]:
condition = param.reactive(True)

Now let's say we want to return either Parameter `a` or `b` depending on whether the condition is True or False. We can simply pass the values to `.where()`:

In [ ]:
p = Parameters(a=1, b=2)

ternary_expr = condition.rx.where(p.param.a, p.param.b)

ternary_expr

Since the initial value is `True` it returns the current value of `a`, which is `1`. However when we set the value to `False` it will return the value of `b`:


In [ ]:
condition.rx.set(False)

ternary_expr.rx.resolve()

Importantly, if we now change `b` the result will be reflected by the expression, reactively unless we explicitly resolve the result:

In [ ]:
p.b = 5

ternary_expr.rx.resolve()

Here the expression value depends only on `b` thanks to the `where` condition, and thus e changes to `a` will no longer trigger any downstream updates until the condition is reversed again.